In [ ]:
import geopandas as gpd

gdf = gpd.read_file('temp/country-borders-raw.geo.jsonseq', engine="pyogrio")
gdf

In [ ]:
gdf.plot(figsize=(20,20))

In [ ]:
for c in gdf.columns:
    if ':' in c:
        continue
    print(f"        '{c}',")

In [ ]:
COUNTRY_TAGS = [
        'boundary',
        'admin_level',
        'name',
        'name:en',
        'ISO3166-1',
        'ISO3166-1:alpha2',
        'ISO3166-1:alpha3',
        'ISO3166-1:numeric',
        'ISO3166-2',
        'alt_int_name',
        'alt_name',
        'wikidata',
        'wikipedia',
        'geometry',
]

In [ ]:
#temp = gdf[COUNTRY_TAGS].dropna(subset='name')
#temp[temp.geom_type == 'MultiPolygon'][20:40]# .plot(figsize=(30,30))
#temp[temp['name'] == 'Isla Tres Cruces']

In [ ]:
#temp = temp[temp.geom_type == 'MultiPolygon']
#temp[temp['ISO3166-1'].isna()]
#temp[temp['name'] == 'Isla Tres Cruces'].plot()

In [ ]:
countries = gdf[COUNTRY_TAGS].dropna(subset='name').dropna(subset='ISO3166-1')
countries

In [ ]:
countries.plot(figsize=(20,20))

In [ ]:
countries.geom_type.describe()

In [ ]:
import json
from tqdm import tqdm

extracts = []

for index in tqdm(range(len(countries))):
    row = countries.iloc[index]
    CC = row['ISO3166-1']
    
    countries.iloc[[index]].to_file(f"extracts/{CC}-borders.geojson")

    extracts.append({
        'output': f'{CC}.pbf',
        'output_format': 'pbf',
        'description': f"{row['name:en']} / {row['name']}",
        'multipolygon': {
            "file_name": f"{CC}-borders.geojson",
            "file_type": "geojson"
        }
    })

config = {
    "directory": "extracts",
    "extracts": extracts
}

fp = open('extracts-config.json', 'wt', encoding='UTF8')
json.dump(config, fp, sort_keys=True, indent=4, ensure_ascii=False)
fp.close()